### Designing a finite state automaton in Python

CS 236 <br>
Fall 2023

Michael A. Goodrich <br>
Brigham Young University <br>
February 16, 2023
***

We'll be creating code to implement the FSA shown below. The FSA accepts all strings that begin with ':-'. The regular expression describing the language recognized by this FSA is :-$I^*$ where $I$ is the set of possible inputs.
![colon-dash FSA](colon-dash-fsa.png)

The tuple defining the finite state automata shown above is (see Def 3 in section 13.3.3)
* A finite set of states $S=\{s_0,s_1,s_2,s_{\rm err}\}$
* A set of input characters $I=\{{\rm any\  keyboard\  character}\}$
* A start state $s_0$
* A set of final or accept states $F=\{s_2\}$
* A transition function $f:S\times I \rightarrow S$
<br>

We will now create a class that implements this FSA
***

In [30]:
class colonDash_FSA:
    def __init__(self):
        ###############################################################
        # Class constructor
        # Define four variables that are used within the FSA, some
        # to make the FSA run and some that help us understand how 
        # the FSA works
        ###############################################################
        self.input_string = ""      # Default empty input
        self.FSA_name = "colon-dash state machine"
        self.num_chars_read = 0
        self.history = []
        ###############################################################
        # Define the start state and the set of accept states
        ###############################################################
        self.start_state = self.S0  # Initialize the starting state
        self.accept_states = set()  # Set of accept states
        self.accept_states.add(self.S2)
        ###############################################################
        # Each state will be assigned its own function
        # The transition function is defined within the state functions
        ###############################################################
        
    def Run(self,input_string):
        ###############################################################
        # This function will be called to make the FSA execute
        # It records the input string,
        # sets the current state to the start state
        # and then calls a state method that returns the next state
        # Each state method accesses the input_string
        ###############################################################
        self.input_string = input_string
        current_state = self.start_state
        while self.num_chars_read < len(self.input_string):
            current_state = current_state()
        if current_state in self.accept_states:
            return True # Accept if the FSA ended in an accept state
        else: 
            return False # Reject if the FSA ended in anything other than an accept state
    
    #########################################################
    # Define each state as a function                       #
    # Within each function, define the transition function. #
    # Each transition function reads the current input and  #
    # choose the next state based on the input              #
    #########################################################
    def S0(self):
        current_input = self.input_string[self.num_chars_read]
        if current_input == ':': 
            next_state = self.S1
            history_message = self.get_History_String("S0",current_input,"S1")
        else: 
            next_state = self.Serr
            history_message = self.get_History_String("S0",current_input,"Serr")
        self.num_chars_read += 1
        self.history.append(history_message)
        return next_state
    def S1(self):
        current_input = self.input_string[self.num_chars_read]
        if current_input == '-': 
            next_state = self.S2
            history_message = self.get_History_String("S1",current_input,"S2")
        else: 
            next_state = self.Serr
            history_message = self.get_History_String("S1",current_input,"Serr")
        self.num_chars_read += 1
        self.history.append(history_message)
        return next_state
    def S2(self):
        current_input = self.input_string[self.num_chars_read]
        next_state = self.S2 # loop in state S2
        history_message = self.get_History_String("S2",current_input,"S2")
        self.num_chars_read += 1
        self.history.append(history_message)
        return next_state
    def Serr(self):
        current_input = self.input_string[self.num_chars_read]
        next_state = self.Serr # loop in state Serr
        history_message = self.get_History_String("Serr",current_input,"Serr")
        self.num_chars_read += 1
        self.history.append(history_message)
        return next_state
    
    ############################
    # Manager functions
    ############################
    def Reset(self):
        self.num_chars_read = 0
        self.history = []
    
    ############################
    # Getters and Setters
    ############################
    def get_Name(self): return self.FSA_name
    def get_History(self): 
        output_string = ""
        for message in self.history:
            output_string = output_string + message + "\n"
        return output_string

    ############################
    # Helper functions
    ############################
    def get_History_String(self,current_state,input,next_state):
        history_message = "From state " + current_state + " read input " + \
            input + " and transitioned to state " + next_state
        return history_message
    

In [31]:
my_FSA = colonDash_FSA()
input_string = ":-"
accept_status = my_FSA.Run(input_string)
print("The output of the", my_FSA.get_Name(), "was", accept_status,"when the input was '",input_string,"'")
print("Trace the following history using the picture of the FSA above:")
print(my_FSA.get_History())


The output of the colon-dash state machine was True when the input was ' :- '
Trace the following history using the picture of the FSA above:
From state S0 read input : and transitioned to state S1
From state S1 read input - and transitioned to state S2



In [32]:
input_string = ":"
my_FSA.Reset()
accept_status = my_FSA.Run(input_string)
print("The output of the", my_FSA.get_Name(), "was", accept_status,"when the input was '",input_string,"'")
print("Trace the following history using the picture of the FSA above:")
print(my_FSA.get_History())

The output of the colon-dash state machine was False when the input was ' : '
Trace the following history using the picture of the FSA above:
From state S0 read input : and transitioned to state S1



In [34]:
input_string = ":::-"
my_FSA.Reset()
accept_status = my_FSA.Run(input_string)
print("The output of the", my_FSA.get_Name(), "was", accept_status,"when the input was '",input_string,"'")
print("Trace the following history using the picture of the FSA above:")
print(my_FSA.get_History())

The output of the colon-dash state machine was False when the input was ' :::- '
Trace the following history using the picture of the FSA above:
From state S0 read input : and transitioned to state S1
From state S1 read input : and transitioned to state Serr
From state Serr read input : and transitioned to state Serr
From state Serr read input - and transitioned to state Serr

